## Matplotlib Animation

In [22]:
import cartopy.crs as ccrs
import cartopy.feature as cfeat
import matplotlib.animation as animation
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib
from celluloid import Camera


#####******************* Inspired by http://tech.weatherforce.org/blog/ecmwf-data-animation/index.html******************########
#####*******************Inspired by https://gist.github.com/ScottWales/5d0ef918d56c4f6f58b73c1d5eabaf9b*******************########

def make_figure(cartopy_projection):
    fig = plt.figure(figsize=(20,15))
    ax = plt.axes(projection=cartopy_projection)
    ax.add_feature(cfeat.LAND)
    #ax.add_feature(cfeat.OCEAN)
    ax.add_feature(cfeat.COASTLINE)
    #ax.add_feature(cfeat.BORDERS, linestyle='dotted')
    return fig, ax


def draw(frame,ds,dsu,dsv,varname,ax,min_value,max_value,N,cmap1,add_colorbar):
        ax.clear()
        ax               = plt.axes(projection=cartopy_projection)
        ax.add_feature(cfeat.LAND)
        
        
        area              = ds[varname]
        frames            = area.time.size
        grid              = area[frame]
        levels1           =np.linspace(min_value,max_value,N)
        
        
        u                  = dsu['uwnd'][frame] 
        v                  = dsv['vwnd'][frame]

        grid.plot(ax=ax, transform=ccrs.PlateCarree(),cmap=cmap1,extend='both',
                            add_colorbar=add_colorbar,levels=levels1,animated=True)
        
        ax.quiver(dsu.lon.values,dsu.lat.values,u.values,v.values,color='black',transform = ccrs.PlateCarree(),animated=True)
        
        ax.set_global()
        ax.coastlines()
        title = u"%s — %s" % (ds[varname].name, str(area.time[frame].values)[:19])
        ax.set_title(title)
        
        return ax

    
def initial():
    return draw(0,ds,dsu,dsv,varname,ax,min_value,max_value,N,cmap1,add_colorbar=True)


def animate(frame):
    return draw(frame,ds,dsu,dsv,varname,ax,min_value,max_value,N,cmap1,add_colorbar=False)


def make_animation(fig,frames,outputname):
    ani = animation.FuncAnimation(fig,animate,frames, interval=1, blit=False,init_func=initial, repeat=False)
    #ani.save(outputname, writer=animation.FFMpegWriter(fps=4))
    ani.save(outputname,writer='imagemagick') 
    plt.close(fig)
    return ani



## BOB

In [25]:
b=xr.open_dataset('/home/cccr/roxy/saranya/DATA_all/mhw_anomalies/v850.anomalies.nc')
c=xr.open_dataset('/home/cccr/roxy/saranya/DATA_all/mhw_anomalies/u850.anomalies.nc')

dsv =b.sel(lat=slice(90,-90),lon=slice(0,120),time=slice('2010-06-26','2010-07-29'),level=850)
dsu =c.sel(lat=slice(90,-90),lon=slice(0,120),time=slice('2010-06-26','2010-07-29'),level=850)
dsu

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 73, lon: 49, time: 34)
Coordinates:
  * time       (time) datetime64[ns] 2010-06-26 2010-06-27 ... 2010-07-29
  * lon        (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 112.5 115.0 117.5 120.0
  * lat        (lat) float32 90.0 87.5 85.0 82.5 ... -82.5 -85.0 -87.5 -90.0
    level      float32 850.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    uwnd       (time, lat, lon) float32 ...
Attributes:
    CDI:            Climate Data Interface version 1.9.6 (http://mpimet.mpg.d...
    Conventions:    CF-1.0
    history:        Fri Mar 20 17:33:36 2020: cdo -O sub /home/cccr/roxy/sara...
    source:         NCEP/DOE AMIP-II Reanalysis (Reanalysis-2) Model
    institution:    National Centers for Environmental Prediction
    title:          Daily NCEP/DOE Reanalysis 2
    comments:       Data is from \nNCEP/DOE AMIP-II Reanalysis (Reanalysis-2)...
    platform:       Model
    dataset_title:  NCEP-DOE AMIP-II Reanalysis
    References:     https://www.esrl.noaa.gov/psd/data/gridded/data.ncep.rean...
    source_url:     http://www.cpc.ncep.noaa.gov/products/wesley/reanalysis2/
    CDO:            Climate Data Operators version 1.9.6 (http://mpimet.mpg.d...

In [26]:
import nclcmaps 
cmap1=nclcmaps.cmap('sunshine_9lev')

a=xr.open_dataset('sst_anom_mhws.nc')
a1 =a.sel(time=slice('2010-06-26','2010-07-29'))


ds = a1*1
varname ='sstanom';
cartopy_projection=ccrs.Orthographic(central_latitude=0.0,central_longitude=75.0)
min_value   =   0.00
max_value   =   0.8
N           =   11
cmap1       =   cmap1
fig,ax      = make_figure(cartopy_projection)
make_animation(fig,frames=ds.time.size,outputname='o1.gif')
# ds.time.size

/home/cccr/roxy/.conda/envs/saranya/lib/python3.7/site-packages/ipykernel_launcher.py:27: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


In [ ]:
a=xr.open_dataset('sst_anom_mhws.nc')
a1 =a.sel(time=slice('2010-06-26','2010-07-29'))
import nclcmaps 
cmap1=nclcmaps.cmap('sunshine_9lev')

In [ ]:
ds = a1*1
varname ='sstanom';
cartopy_projection=ccrs.Orthographic(central_latitude=0.0,central_longitude=75.0)
min_value   =   0.00
max_value   =   0.8
N           =   11
cmap1       =   cmap1
fig,ax      = make_figure(cartopy_projection)
area,frames,ds =read(ds,varname)
make_animation(fig,frames,'output.mp4')

## AS

In [ ]:
a1 =a.sel(time=slice('2017-04-19','2017-06-02'))

ds = a1*1
varname ='sstanom';
cartopy_projection=ccrs.Orthographic(central_latitude=0.0,central_longitude=75.0)
min_value   =   0.00
max_value   =   0.8
N           =   11
cmap1       =   cmap1
fig,ax      = make_figure(cartopy_projection)
area,frames,ds =read(ds,varname)
make_animation(fig,frames,'output_AS.mp4')